# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Read CSV and save to dataframe
city_data = pd.read_csv(f"Resources/citydata.csv")
city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,berberati,4.2612,15.7922,67.71,95,100,4.74,CF,1619650882
1,rikitea,-23.1203,-134.9692,76.21,67,43,10.49,PF,1619650882
2,mednogorsk,51.4128,57.5950,43.21,82,75,1.19,RU,1619650882
3,colwyn bay,53.2948,-3.7267,46.40,71,98,9.22,GB,1619650883
4,lorengau,-2.0226,147.2712,83.62,74,27,5.55,PG,1619650759
...,...,...,...,...,...,...,...,...,...
543,unicov,49.7709,17.1214,45.00,83,83,2.84,CZ,1619651070
544,mount pleasant,32.7941,-79.8626,75.20,69,1,13.80,US,1619651071
545,juneau,58.3019,-134.4197,46.00,65,90,11.50,US,1619650942
546,stornoway,58.2093,-6.3865,39.20,87,97,6.91,GB,1619651072


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)
city_data = city_data.dropna()

# Set locations
locations = city_data[["Lat", "Lng"]].astype(float)

# Convert humidity to float and store
humidity = city_data["Humidity"].astype(float)

In [6]:
# Create a poverty Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# My conditions: 
    # Temperature between 68 and 86 Farenheit
    # Cloudiness of 40% or less
    # Humidity between 30% and 50%
cond1 = city_data[city_data['Max Temp'] <= 86]
cond1 = cond1[cond1['Max Temp'] >= 68]
cond2 = cond1[cond1['Cloudiness'] <= 40]
cond3 = cond2[cond2['Humidity'] <= 50]
cities_criteria = cond3[cond3['Humidity'] >= 30]
cities_criteria

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,bethel,41.3712,-73.4140,75.20,43,1,9.22,US,1619650887
32,anderson,34.5334,-82.6499,82.00,47,1,8.05,US,1619650892
51,avera,33.1940,-82.5271,82.40,47,1,6.91,US,1619650899
133,pacific grove,36.6177,-121.9166,69.80,46,1,6.91,US,1619650929
140,qurayyat,31.3318,37.3428,68.00,30,0,11.50,SA,1619650931
232,kogon,39.7228,64.5517,71.60,35,0,2.30,UZ,1619650752
243,mount isa,-20.7333,139.5000,71.60,37,0,4.61,AU,1619650821
314,san juan,-31.5375,-68.5364,72.14,32,0,7.92,AR,1619650988
328,umm lajj,25.0213,37.2685,81.05,38,0,6.22,SA,1619650993
341,pueblo nuevo,23.3833,-105.3833,69.51,41,10,9.35,MX,1619650998


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Get cities coordinates and look for airports
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
cities_hotels = cities_criteria[['City','Lat','Lng']]
hotels = []
for index, row in cities_hotels.iterrows():
    lats = cities_hotels['Lat'][index]
    lngs = cities_hotels['Lng'][index]
    params = {
        "location": f"{lats},{lngs}",  
        "rankby": "distance",
        "keyword": "hotel",
        "key": g_key,}
    # Get airports
    response = requests.get(base_url, params=params).json()
    results = response['results']
    hotels.append(results[0]['name'])
hotels

['Hampton Inn Danbury',
 'Red Roof Inn & Suites Anderson, SC',
 "Boss's RV Park",
 'Pacific Grove Inn',
 'Dana North Apartment',
 'Zargaron Plaza',
 'Travellers Haven',
 'Hotel San Francisco',
 'Juman Hotel',
 'Cabañas La Cueva Aventura Extrema',
 'Saint James Palace Hotel',
 'Hotel RIKMAN Continental',
 'Country House General dacha',
 'Astorga Park Hotel']

In [9]:
# Add to dataframe
hotel_df = pd.DataFrame(cities_criteria)
hotel_df['Hotel Name'] = hotels
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
18,bethel,41.3712,-73.4140,75.20,43,1,9.22,US,1619650887,Hampton Inn Danbury
32,anderson,34.5334,-82.6499,82.00,47,1,8.05,US,1619650892,"Red Roof Inn & Suites Anderson, SC"
51,avera,33.1940,-82.5271,82.40,47,1,6.91,US,1619650899,Boss's RV Park
133,pacific grove,36.6177,-121.9166,69.80,46,1,6.91,US,1619650929,Pacific Grove Inn
140,qurayyat,31.3318,37.3428,68.00,30,0,11.50,SA,1619650931,Dana North Apartment
232,kogon,39.7228,64.5517,71.60,35,0,2.30,UZ,1619650752,Zargaron Plaza
243,mount isa,-20.7333,139.5000,71.60,37,0,4.61,AU,1619650821,Travellers Haven
314,san juan,-31.5375,-68.5364,72.14,32,0,7.92,AR,1619650988,Hotel San Francisco
328,umm lajj,25.0213,37.2685,81.05,38,0,6.22,SA,1619650993,Juman Hotel
341,pueblo nuevo,23.3833,-105.3833,69.51,41,10,9.35,MX,1619650998,Cabañas La Cueva Aventura Extrema


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(
    locations, 
    info_box_content=[f"Name: {hotel_df['Hotel Name'][index]} | City: {hotel_df['City'][index]} | Country: {hotel_df['Country'][index]}"
                      for index, row in hotel_df.iterrows()]
)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))